In [1]:
# Package Setup
import Pkg;

# Julia Packages
using Distributions
using LinearAlgebra
using Statistics
using PGFPlots

# Temporary for now
using JuMP
using Gurobi

# Allow for easy updating
using Revise

# Satellite Dynamics Packages
using SatelliteDynamics

# Load SatelliteTasking - Reclone to keep version current
using SatelliteTasking
using SatelliteTasking.SatellitePlanning
using SatelliteTasking.Analysis

┌ Info: Recompiling stale cache file /Users/deddy/.julia/compiled/v1.1/SatelliteTasking/HFrko.ji for SatelliteTasking [30f2e302-0ae1-11e9-1cda-0f4324764595]
└ @ Base loading.jl:1184


In [2]:
# Load Image Data

# Load test images
# @time images = load_images("../data/landsat_test_150.json", collect_duration=5.0);
# @time images = load_images("../data/landsat_test_300.json", collect_duration=5.0);
@time images = load_images("../data/landsat_test_600.json", collect_duration=5.0);
num_images = length(images)

println("Loaded $num_images images.")

  0.425053 seconds (1.25 M allocations: 64.979 MiB, 5.04% gc time)
Loaded 600 images.


In [3]:
# Simulation Definition

# Configure simulation
epc0 = Epoch(2019, 1, 1, 0, 0, 0, tsys=:UTC) # Start of time span
epcf = Epoch(2019, 1, 2, 0, 0, 0, tsys=:UTC) # End of simulation time span

# Set Simulation Time Step
timestep = 1

# Define Satellite Orbit
oe   = [R_EARTH + 500e3, 0, 90.0, 0, 0, 0]
eci0 = sOSCtoCART(oe, use_degrees=true)

# UNCOMMENT AND PASS AS KEYWORD TO ORBIT TO OVERRIDE DEFAULT VALUE
# # Spacecraft model parameters
# mass      = 100.0 # [kg]
# area_drag = 1.0   # [m^2]
# coef_drag = 2.3
# area_srp  = 1.0   # [m^2]
# coef_srp  = 1.8

# # Define force model parameters
# n_grav = 20
# m_grav = 20
# drag   = true
# srp    = true
# sun    = true
# moon   = true
# relativity = true

true

In [4]:
# Simulate Orbit

# Simulate True Orbit
@time orbit = Orbit(epc0, epcf, eci0, timestep=timestep)

 74.888525 seconds (1.38 G allocations: 34.545 GiB, 6.75% gc time)


Orbit(0)

In [5]:
# Compute Opportunities & Collects

# Compute Opportunities
@time original_opportunities = find_all_opportunities(orbit, images, sort=true);

# Compute Collects
@time opportunities = split_opportunities(original_opportunities, 3);

# Compute feasible collects
image_opps   = group_image_opportunities(opportunities) # Group collects by image
num_feasible = 0
for opps in keys(image_opps)
    if length(image_opps[opps]) > 0
        num_feasible += 1
    end
end
pct_feasible = num_feasible/num_images*100

println("$num_feasible out of $num_images images have collection opportunities.")

 51.656006 seconds (674.43 M allocations: 71.150 GiB, 13.34% gc time)
  0.175303 seconds (362.09 k allocations: 18.727 MiB, 2.16% gc time)
552 out of 600 images have collection opportunities.


In [6]:
# Graph Planning

@time graph = sp_construct_graph(opportunities, Function[constraint_agility_single_axis], horizon=0.0);

@time graph_path, graph_reward, graph_images = sp_solve_graph(graph, allow_repeats=false);

println("Total planning reward: $graph_reward")
println("Number of images collected: $(length(graph_images))/$num_images, $(length(graph_images)/num_images*100)")
println("Number of feasible images collected: $(length(graph_images))/$num_feasible, $(length(graph_images)/num_feasible*100)")


graph_feasible = sp_check_feasibility(graph_path, Function[constraint_agility_single_axis]);
println("GRAPH Constraint Check: $(graph_feasible ? "PASS" : "FAIL")")

  6.544192 seconds (21.59 M allocations: 1.569 GiB, 12.42% gc time)
  7.420195 seconds (70.13 M allocations: 5.499 GiB, 20.55% gc time)
Total planning reward: 525.0
Number of images collected: 525/600, 87.5
Number of feasible images collected: 525/552, 95.1086956521739
GRAPH Constraint Check: PASS


In [7]:
# MILP planning
@time milp_path, milp_reward, milp_images = sp_milp_policy(opportunities, Function[constraint_agility_single_axis], horizon=0.0, allow_repeats=false)

println("Total planning reward: $milp_reward")
println("Number of images collected: $(length(milp_images))/$num_images, $(length(milp_images)/num_images*100)")
println("Number of feasible images collected: $(length(milp_images))/$num_feasible, $(length(milp_images)/num_feasible*100)")

milp_feasible = sp_check_feasibility(milp_path, Function[constraint_agility_single_axis])
println("MILP Constraint Check: $(milp_feasible ? "PASS" : "FAIL")")

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Optimize a model with 70959 rows, 5575 columns and 146389 nonzeros
Variable types: 0 continuous, 5575 integer (5575 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 520.0000000
Presolve removed 65950 rows and 3215 columns
Presolve time: 0.75s
Presolved: 5009 rows, 2360 columns, 56526 nonzeros
Found heuristic solution: objective 531.0000000
Variable types: 0 continuous, 2360 integer (2360 binary)

Root relaxation: objective 5.430000e+02, 990 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                     543.0000000 2671.00000   392%     -    0s
     0     0          -    0       543.00000  543.0000

In [9]:
# MDP planning

collect_probs = Dict(o => 1.0 for o in opportunities)

@time mdp_path, mdp_reward, mdp_images = mdp_forward_search(opportunities, Function[constraint_agility_single_axis], 
                                                            images, collect_probs, horizon=4000, gamma=0.7, search_depth=1);

println("Total planning reward: $mdp_reward")
println("Number of images collected: $(length(mdp_images))/$num_images, $(length(mdp_images)/num_images*100)")
println("Number of feasible images collected: $(length(mdp_images))/$num_feasible, $(length(mdp_images)/num_feasible*100)")

mdp_feasible = sp_check_feasibility(mdp_path, Function[constraint_agility_single_axis])
println("MDP Constraint Check: $(mdp_feasible ? "PASS" : "FAIL")")

Number of all locations: 600, Opportunityed: 475, Missed: 125
  7.641543 seconds (24.89 M allocations: 1.879 GiB, 6.24% gc time)
Total planning reward: 475.0
Number of images collected: 475/600, 79.16666666666666
Number of feasible images collected: 475/552, 86.05072463768117
MDP Constraint Check: PASS
